In [ ]:
import argparse
import time
from pathlib import Path

import cv2
import numpy as np
import torch
import torch.backends.cudnn as cudnn

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path, save_one_box

from utils.plots import colors, plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

from matplotlib import pyplot as plt
from utils.datasets import letterbox

In [ ]:
weights = "./weights/best.pt"
imgsz = 320

view_img = True
save_txt = False
save_img= False

webcam = False
save_dir = "./runs/"

device = "cpu"

iou_thres = 0.45
conf_thres = 0.4
classes = 0,1,2
agnostic_nms = True
max_det = 1000

line_thickness = 2

In [ ]:
model = attempt_load(weights, map_location=device)  # load FP32 model
stride = int(model.stride.max())  # model stride
imgsz = check_img_size(imgsz, s=stride)  # check img_size
names = model.module.names if hasattr(model, 'module') else model.names  # get class names

In [ ]:
def detect_yolo(image1):
    
    with_msk = 0
    without_msk = 0
    improper = 0
    
    count_txt = " "
    alert = "No Covid Violation"
    
    # for path, img, im0s, vid_cap in dataset:
    img0 = image1
    # Padded resize
    img = letterbox(img0, imgsz, stride=stride)[0]

    # Convert
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)


    img = torch.from_numpy(img).to(device)
    img = img.float()  # uint8 to fp32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    pred = model(img, augment = False)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms,
                                       max_det=max_det)
    
    for i, det in enumerate(pred):
        im0 = img0.copy()

        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
        imc = im0  # for opt.save_crop

        if len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            # Print results
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class

                print(n)

            # Write results
            for *xyxy, conf, cls in reversed(det):

                c = int(cls)  # integer class
                label = names[c] 
                plot_one_box(xyxy, im0, label=label, color=colors(c, True), line_thickness = line_thickness)
                
                if c == 0 :
                    with_msk += 1
                elif c == 1:
                    without_msk += 1
                else:
                    improper += 1
                    
                count_txt = "With Mask :" + str(with_msk) + "; Without Mask :" + str(without_msk) + "; ImproperMask :" + str(improper) + ";"
                
                if without_msk or improper:
                    alert = "Covid Protocol Violation : " + str(without_msk+improper)  +" Individuals"
            
    return  im0, count_txt, alert

In [ ]:
def detect(webcam, algo):
    
    if algo == "Yolo":
        image2, count_txt, alert = detect_yolo(webcam)
    else:
        image2, count_txt, alert = detect_yolo(webcam)

    return image2, count_txt, alert

In [ ]:
import gradio as gr

img = gr.inputs.Image()

title = "Face Mask - Detection"

algo = gr.inputs.Radio(["Yolo", "Other"], label="Choose Algorithm")
webcam = gr.inputs.Image(shape=(200, 200), source="webcam")

image2 = gr.outputs.Image(type="numpy", label="Output")

count_txt = gr.outputs.Textbox(type="auto", label="Comment")
alert = gr.outputs.Textbox(type="auto", label="Alert")

gr.Interface(detect, inputs = [webcam, algo],   
             outputs = [image2, count_txt, alert], 
             title=title,
             live=True).launch(share=True)